In [1]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import os
import shutil
import re
from geopy.geocoders import Nominatim
import folium
from sklearn.manifold import TSNE
import plotly.graph_objects as go
import os
import matplotlib.pyplot as plt
import plotly

In [2]:
len(os.listdir('Flags'))

206

In [3]:
for file_name in os.listdir('Flags'):
    print(file_name[:-4])

Afghanistan
Albania
Algeria
Andorra
Angola
Antigua_and_Barbuda
Argentina
Armenia
Artsakh
Australia
Austria
Azerbaijan
Bahamas
Bahrain
Bandera_de_España
Bangladesh
Barbados
Belarus
Belgium
Belize
Benin
Bhutan
Bolivia
Bosnia_and_Herzegovina
Botswana
Brazil
Brunei
Bulgaria
Burkina_Faso
Burundi
Cambodia
Cameroon
Canada
Cape_Verde
Central_African_Republic
Chad
Chile
Colombia
Comoros
Cook_Islands
Costa_Rica
Croatia
Cuba
Cyprus
Czech_Republic
Côte_d'Ivoire
Democratic_Republic_of_Congo
Denmark
Djibouti
Dominica
Dominican_Republic
East_Timor
Ecuador
Egypt
El_Salvador
Equatorial_Guinea
Eritrea
Estonia
Eswatini
Ethiopia
Federated_States_of_Micronesia
Fiji
Finland
France
Gabon
Georgia
Germany
Ghana
Greece
Grenada
Guatemala
Guinea-Bissau
Guinea
Guyana
Haiti
Honduras
Hungary
Iceland
India
Indonesia
Iran
Iraq
Ireland
Israel
Italy
Jamaica
Japan
Jordan
Kazakhstan
Kenya
Kiribati
Kosovo
Kuwait
Kyrgyzstan
Laos
Latvia
Lebanon
Lesotho
Liberia
Libya
Liechtenstein
Lithuania
Luxembourg
Madagascar
Malawi
Malays

In [4]:
def image_feature(direc):
    model = InceptionV3(weights='imagenet', include_top=False)
    features = [];
    img_name = [];
    file_name = [];
    for i in tqdm(direc):
        fname = 'Flags/'+i
        img=image.load_img(fname,target_size=(120,120))
        x = img_to_array(img)
        x = np.expand_dims(x,axis=0)
        x = preprocess_input(x)
        feat = model.predict(x)
        feat = feat.flatten()
        features.append(feat)
        img_name.append(i.replace("_"," ")[:-4])
        file_name.append(fname)
    return features,img_name,file_name

In [5]:
img_path=os.listdir('Flags')
img_features,img_name,file_name=image_feature(img_path)

  0%|          | 0/206 [00:00<?, ?it/s]

1/1 [==============================] - 0s 78ms/step


D:\Software\Anaconda\lib\site-packages\PIL\Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


1/1 [==============================] - 0s 88ms/step


In [7]:
df = pd.DataFrame(img_name,columns=['Country'])
df['File'] = file_name
df["clusterid"] = clusters.labels_
df['latitude'] = None
df['longitude'] = None
geolocator = Nominatim(user_agent="geoapiExercises")
for i in tqdm(range(len(df))):
    location = geolocator.geocode(df.loc[i,'Country'])
    if location:
        df.loc[i,'latitude'] = location.latitude
        df.loc[i,'longitude'] = location.longitude
    else:
        print(f"{df.loc[i,'Country']} not found")

df

  0%|          | 0/206 [00:00<?, ?it/s]

,Country,File,clusterid,latitude,longitude
0,Afghanistan,Flags/Afghanistan.png,4,33.768006,66.238514
1,Albania,Flags/Albania.png,2,41.000028,19.999962
2,Algeria,Flags/Algeria.png,0,28.000027,2.999983
3,Andorra,Flags/Andorra.png,9,42.540717,1.573203
4,Angola,Flags/Angola.png,9,-11.877577,17.569124
...,...,...,...,...,...
201,Venezuela,Flags/Venezuela.png,0,8.001871,-66.110932
202,Vietnam,Flags/Vietnam.png,1,15.926666,107.965086
203,Yemen,Flags/Yemen.png,0,16.347124,47.891527
204,Zambia,Flags/Zambia.png,0,-14.518912,27.558988


In [8]:
df.to_csv("flag_cluster.csv", index=False)

In [6]:
k = 10
clusters = KMeans(k, random_state = 40)
clusters.fit(img_features)

KMeans(n_clusters=10, random_state=40)

In [9]:
df = pd.read_csv("flag_cluster.csv")

In [10]:
latitude = df['latitude'].mean()
longitude = df['longitude'].mean()
map = folium.Map(location=[latitude, longitude], zoom_start=3)

# Iterate over each row in the dataframe and add a marker for each place
for i in range(len(df)):
    latitude = df.loc[i,'latitude']
    longitude = df.loc[i,'longitude']
    name = df.loc[i,'Country']
    cluster = df.loc[i,'clusterid']
    colors = {0:'black', 1:'lightblue', 2:'red', 3:'blue', 4:'green', 5:'yellow', 6:'orange', 7:'pink', 8:'purple',9:'gray'}
    folium.Marker(
        location=[latitude, longitude],
        popup=name,
        icon=folium.Icon(color=colors[cluster])
    ).add_to(map)
    folium.features.ImageOverlay(
    image=df.loc[i,'File'],
    bounds=[[51.49, -0.08], [51.51, -0.06]],
    opacity=0.6
    )

# Display the map
map.save('map.html')
map

C:\Users\mohan\AppData\Local\Temp\ipykernel_14028\521594609.py:15: UserWarning: color argument of Icon should be one of: {'darkgreen', 'lightblue', 'gray', 'darkblue', 'darkpurple', 'beige', 'white', 'orange', 'cadetblue', 'black', 'darkred', 'green', 'lightgray', 'pink', 'purple', 'lightgreen', 'blue', 'red', 'lightred'}.
  icon=folium.Icon(color=colors[cluster])


In [11]:

# set seed for reproducible results
seed = 10
np.random.seed(seed)

# run tsne
n_components = 2
tsne = TSNE(n_components)
tsne_result = tsne.fit_transform(img_features)


def scale_to_01_range(coordinates,scale = 5000):
    min_val = np.min(coordinates)
    max_val = np.max(coordinates)
    return (coordinates - min_val)*scale / (max_val - min_val)

# scale and move the coordinates so they fit [0; 1] range
tx = scale_to_01_range(tsne_result[:,0])
ty = scale_to_01_range(tsne_result[:,1])

D:\Software\Anaconda\lib\site-packages\sklearn\manifold\_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
D:\Software\Anaconda\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


In [20]:

# Set the directory containing the images
directory = "Flags"

# Read the x and y coordinates
x = list(tx)
y = list(ty)

# Create a figure
fig = go.Figure()


# Iterate through the images in the directory
for i, filename in enumerate(os.listdir(directory)):
    # Read the image file
    img_path = './'+directory+'/'+filename
    # Add the image to the plot
    img = plt.imread(img_path)
    
    fig.add_layout_image(
        dict(
            source=img_path,
            xref="x",
            yref="y",
            x=x[i],
            y=y[i],
            sizex=img.shape[1],
            sizey=img.shape[0],
            sizing="stretch",
            layer="below"
        )
    )

    
fig.add_trace(go.Scatter(x=x,y=y,mode='markers',marker=dict(color=clusters.labels_),text=img_name))
    
# Set the plot background color to white
fig.update_layout(
    paper_bgcolor='white',
    plot_bgcolor='white'
)

# set the x and y range
fig.update_xaxes(range=[min(tx), max(tx)])
fig.update_yaxes(range=[min(ty), max(ty)])

# Remove the x and y axis
fig.update_xaxes(showgrid=False, showline=False, showticklabels=False)
fig.update_yaxes(showgrid=False, showline=False, showticklabels=False)

fig.update_yaxes(scaleanchor="x",scaleratio=1)
plotly.offline.plot(fig,filename = "index.html")
# Show the plot
fig.show()
